In [1]:
import pandas as pd
import seaborn as sns
import os
import numpy as np

In [2]:
path_to_N200 = '../processing_EEG/N200_estimates/N200_single_trial/'
list_dir = os.listdir(path_to_N200)
list_ = []
for f in list_dir:
    df_ = pd.read_table(path_to_N200+f, header=0, sep=',',index_col=0)
    df_['participant'] = f.split('_')[0]
    list_.append(df_)
df = pd.concat(list_, ignore_index=True)

In [3]:
EMG_df = pd.read_csv('../processing_EMG/clean_df.csv')

In [8]:
## Fix the wrong labeling
df.SAT = df.apply(lambda row: 'accuracy' if row['expdResp'] == 'accuracy' else row['SAT'], axis = 1)
df.expdResp = df.apply(lambda row: 'right' if row['expdResp'] == 'accuracy' else row['expdResp'], axis = 1)
df.FC = df.apply(lambda row: 2.0 if row['FC'] == 'high' else 1.0, axis = 1)

In [9]:
## Fix the trial ordering problem after merging the two .bdf during EMG extraction files for S3
EMG_df.loc[EMG_df.participant == "S3", 'trial'] =  np.append(np.linspace(533, 2439, 2440-533),np.linspace(0,532, 533))

In [10]:
data = pd.merge(df, EMG_df, on=['participant','trial','SAT','expdResp','contrast','FC'])

In [11]:
(len(EMG_df)- len(data))/len(EMG_df)# Difference between both datasets = EEG artifacts

0.059122132457769906

In [12]:
len(data)

43557

In [13]:
data_full = data.copy()
data.to_csv('df_full.csv')

In [14]:
data.head()

,FC,SAT,expdResp,contrast,N200,trial,participant,Unnamed: 0,givenResp,response,rt,prmt,mt,trialType,EMG_sequence
0,1.0,accuracy,right,3,234.0,0,S10,2448,right,1,733.398438,631.835935,101.562503,SA,CR
1,1.0,accuracy,right,1,133.0,1,S10,2449,right,1,597.656250,508.789067,88.867183,SA,CR
2,1.0,accuracy,right,1,168.0,2,S10,2450,right,1,531.250000,458.007810,73.242190,SA,CR
3,1.0,accuracy,left,1,143.0,3,S10,2451,left,1,502.929688,430.664065,72.265622,SA,CR
4,1.0,accuracy,left,3,161.0,4,S10,2452,left,1,984.375000,893.554690,90.820310,SA,CR


Computing decision time

In [15]:
data['dt'] = data['prmt'] - data['N200']

Checking rejection criterion

In [16]:
print(len(data[data.rt > 1500].trial))#PMT over 1.5s
print(len(data[data.rt > 1500].trial)/len(data.trial) )

470
0.010790458479693276


In [17]:
print(len(data[data.trialType == "UT"].trial))#EMG unmarked trial (e.g. tonic noise)
print(len(data[data.trialType == "UT"].trial)/len(data.trial) )

1642
0.03769772941203481


In [18]:
print(len(data[data.trialType == "MA"].trial))#Multiple activity trials
print(len(data[data.trialType == "MA"].trial)/len(data.trial) )

7039
0.1616043345501297


In [19]:
print(len(data[~np.isfinite(data.N200)].trial))#N200 not at window boundaries
print(len(data[~np.isfinite(data.N200)].trial)/len(data.trial) )

9823
0.22552058222558946


In [20]:
print(len(data[data.dt <= 0].trial))#Negative decision time
print(len(data[data.dt <= 0].trial)/len(data.trial) )

1775
0.04075119957756503


In [21]:
data[data.dt <= 0].response.mean()#Computing accuracy for negative DT trials

0.5521126760563381

## Applying criteria

In [35]:
data = data[data.rt <= 1500]
data = data[(data.trialType == 'SA')]
data = data[np.isfinite(data.N200)]
data = data[data.dt > 0].reset_index(drop=True)

In [36]:
print(len(data.trial))
print(len(data.trial)/(2448*19))

25288
0.543687650498796


In [38]:
data.to_csv('data_modelling_ready.csv')